# Methods used in several Notebooks

In [1]:
#import pandas as pd #required for most of the methods below

In [2]:
#Pathway of the folder of this file
%pwd

'/Users/mariekececilia/Documents/master_thesis_code'

## Load datasets

In [3]:
#Load both datasets (transcriptomics/features and fluxomics/targets) from Gerosa
def load_gerosa():
    data_folder_path = '/Users/mariekececilia/Documents/master_thesis_code/Data/'
    transcriptomics_path = data_folder_path + 'gerosa_2015_transcriptomics.csv';
    fluxomics_path = data_folder_path + 'gerosa_2015_fluxomics.csv'; 
    
    # Load the datasets with the first columns (containing the description of the rows) as index and
    # transpose it, because want conditions as observations and measurments as features/target
    transcriptomics = pd.read_csv(transcriptomics_path, index_col=0).transpose();
    fluxomics = pd.read_csv(fluxomics_path, index_col=0).transpose();
    
    return [transcriptomics, fluxomics]

#------------------------------------------------------------------------------
#Load both datasets (transcriptomics/features and fluxomics/targets) from Ishii
def load_ishii():
    data_folder_path = '/Users/mariekececilia/Documents/master_thesis_code/Data/'
    transcriptomics_path = data_folder_path + 'ishii_2007_transcriptomics.csv';
    fluxomics_path = data_folder_path + 'ishii_2007_fluxomics.csv'; 
    
    # Load the datasets with the first columns (containing the description of the rows) as index and
    # transpose it, because want conditions as observations and measurments as features/target
    transcriptomics = pd.read_csv(transcriptomics_path, index_col=0).transpose();
    fluxomics = pd.read_csv(fluxomics_path, index_col=0).transpose();
    
    return [transcriptomics, fluxomics]

In [4]:
#Split Ishii into two separate dataframes, one for each perturbation (growth rate and knockouts, the baseline WR_0.2h-1 is included in both)
def split_ishii(df):
    dilution_perturbations = ['WT_0.7h-1','WT_0.5h-1','WT_0.4h-1','WT_0.1h-1'];
    baseline = 'WT_0.2h-1'
    dilution = df.loc[dilution_perturbations + [baseline]]
    knockout = df.drop(dilution_perturbations)
    return [dilution, knockout]

## Preprocessing

### Clean data (remove redundant and unfit columns)

In [5]:
#remove duplicated genes and return dictionary of duplicates
#(if any duplicated gene is shown to be important, the genes with the same measurements can be found using the returned dicitonary)
def clean_gexp_g(df):
    #locate all duplicates first to make it more efficient 
    duplicates = df.T[df.T.duplicated(keep = False)].T
    #group the duplicates
    equal_dict = get_equal_groups(duplicates)
    #remove all but one from each group, i.e. remove all values:
    extras = []
    for key, value in equal_dict.items():
        extras = extras + value
    df = df.drop(extras, axis = 'columns')
    #return dict of groupings so that the duplicates can easily be found later in the analysis
    return df, equal_dict

#Clean Gerosa: remove external reactions, constant reactions, reactions with many null-entries and duplicated reactions
#return duplicates so that reactions that would give the same results can be looked up
def clean_gerosa(df):
    df = remove_ex_constant_zero(df)
    df, equal_dict = remove_extras(df) 
    return df, equal_dict

#Clean Ishii: remove external reactions, constant reactions, reactions with many null-entries and duplicated reactions
#return duplicates so that reactions that would give the same results can be looked up
def clean_ishii(df):
    df = df.drop('R_GLCptspp', axis = 1) #also considered external
    df = remove_ex_constant_zero(df)
    df, equal_dict = remove_extras(df)
    return df, equal_dict
    
# -----------------------------------------------------------------
#remove exchange reactions, constant reactions and reactions with many null-entries
def remove_ex_constant_zero(df):
    
    #remove exchange reactions:
    ex = get_ex(df)
    df = df.drop(ex, axis = 1)
    
    #remove constant reactions:
    constant = df.nunique()[df.nunique() == 1].index.tolist()
    df = df.drop(constant, axis = 1)
    
    #remove reactions where more than half of the entries are zero
    mostly_zero = get_mostly_zero(df)
    df = df.drop(mostly_zero, axis = 1)
    return df

# -----------------------------------------------------------------
#get exchange reactions
def get_ex(df):
    ex = []
    for reaction in df.columns:
        if '_EX_' in reaction:
            ex.append(reaction)
    return ex

# -----------------------------------------------------------------
#get reactions where more than half of the entries are zero
def get_mostly_zero(df):
    total = df.shape[0]
    mostly_zero = []
    for reaction in df.columns:
        if ((df[reaction] == 0).sum() > (total/2)):
            mostly_zero.append(reaction)
    return mostly_zero

# -----------------------------------------------------------------
# remove duplicates and return a dictionary of the them
def remove_extras(df):
    equal_dict = get_equal_or_mirrored_groups(df)
    for key, value in equal_dict.items():
        df = df.drop(value, axis = 1)
    return df, equal_dict

# -----------------------------------------------------------------
#                FIND AND GROUP DUPLICATES
# -----------------------------------------------------------------
"""
Identify columns in a dataframe with equal/mirrored (multiplied by -1) 
measurements and group them. The key for each group is the first label
in alphabetical order, the remaining labels are put in the value set. 
"""
def get_equal_or_mirrored_groups(df): 
    
    #storage
    group_dict = {}
    
    #loop through all pairs of columns
    labels = df.columns.tolist()    
    for i in range(len(labels)):
        L1 = labels[i]
        for j in range(i+1, len(labels)):
            #break the loop when at the end
            if i == len(labels)-1:
                break
            L2 = labels[j]
            
            #compare measurements (sum and differences)
            zero_if_equal = pd.Series(df[L1]-df[L2]) 
            zero_if_mirrored = pd.Series(df[L1]+df[L2])            
            if ((zero_if_equal.sum() == 0) | (zero_if_mirrored.sum() == 0)):
                
                #register/add to relationship group                 
                group_dict = update_groups(L1, L2, group_dict) 
                
    return sort_dict_pairs(group_dict)

#------------------------------------------------------------------------------
"""
Identify columns in a dataframe with equal measurements and group them. 
The key for each group is the first label in alphabetical order, the 
remaining labels are put in the value set. 
"""
def get_equal_groups(df): 
    
    #storage
    group_dict = {}
    
    #loop through all pairs of columns
    labels = df.columns.tolist()    
    for i in range(len(labels)):
        L1 = labels[i]
        for j in range(i+1, len(labels)):
            #break the loop when at the end
            if i == len(labels)-1:
                break
            L2 = labels[j]
            
            #create a series of their differences
            zero_if_equal = pd.Series(df[L1]-df[L2]) 
            
            #check sum of the series to identify those that are equal
            if (zero_if_equal.sum() == 0):
                
                #register/add to relationship group                 
                group_dict = update_groups(L1, L2, group_dict) 
                
    return sort_dict_pairs(group_dict)

#------------------------------------------------------------------------------
"""
Add overlapping relationships to an exisiting dictionary of relationships.
The key of a group is random among its members; the first instance of a group 
is mapped to a set of the other instances 
"""
def update_groups(label_1, label_2, group_dict):
    
    #if one is registerede before, add the other in the correct group
    if label_1 in group_dict.keys():
        group_dict[label_1].add(label_2)
    elif label_2 in group_dict.keys():
        group_dict[label_2].add(label_1)
    
    #if neither are reigstered as keys, check if they are registered as 
    #a value of a third key
    else:
        match = 'NaN'
        # loop through all value sets and their corresponding key
        for value, key in zip(group_dict.values(), group_dict.keys()):
            if (label_1 in value or label_2 in value):
                match = key
                break
        #register a new relationship if no match was found
        if match == 'NaN':
            group_dict[label_1] = set([label_2])
        #otherwise, add the new value to the value set of the match
        else:
            #one is added previously but that does not matter since sets 
            #only accept new values 
            group_dict[match].add(label_1)
            group_dict[match].add(label_2)
                    
    return group_dict

#------------------------------------------------------------------------------
"""
Sort a dictionary that groups realtionships so that the key for each group 
is the first label in alphabetical order, while the remaining labels are in 
the value set. 
"""
def sort_dict_pairs(my_dict):
    new_dict = {}
    for key, value in my_dict.items():
        value.add(key)
        temp = sorted(value)
        new_key = temp.pop(0)
        new_dict[new_key] = temp
    return new_dict

### Standardize data (center and scale to unit variance)

In [6]:
#from sklearn.preprocessing import StandardScaler

'''
Wrapper method which returns the scaled entries (mean = 0, variance = 1) by StandardScaler in a dataframe 
NB! sklearn uses numpy so the standardization uses a different ddof than the default in pandas. Therefore
the variance of the returned features is not 1 if you calculate it with the pandas default, but it is if 
you set the ddof parameter to 0.
'''
def standardize(df):
    scaler = StandardScaler()
    scaled_df = StandardScaler().fit_transform(df)
    scaled_df = pd.DataFrame(scaled_df, columns = df.columns, index = df.index)
    return(scaled_df)

## Explore data

### Intersecion

In [7]:
#Get the intersection of columns of two dataframes
def get_intersection(df1, df2):
    intersect = []
    for col in df1.columns:
        if col in df2.columns:
            intersect.append(col)
    return intersect

### Correlations

In [8]:
#from scipy.stats import spearmanr

"""
Calculate Spearman's correlation rank between all pairs of columns from two dataframes. 
Returns a dataframe with the labels, the rank, and, if chosen, the p-value 
-> set column_names accordingly. 
"""
def get_all_correlations(df_1, df_2, column_names = ['col_1', 'col_2', 'r'], p_val = False):
    
    corr = []
    
    for col_1 in df_1.columns:
        for col_2 in df_2.columns:
            
            r, p = spearmanr(df_1[col_1], df_2[col_2])
            corr.append((col_1, col_2, r, p)) if (p_val) else corr.append((col_1, col_2, r))
            
    corr_df = pd.DataFrame(corr, columns = column_names);
    return corr_df

### Convert gene b-numbers to common names 

Exported table from https://biocyc.org/group?id=biocyc13-24029-3731963750 at 23.05.2022
<br> 4665 rows of all-genes from Escherichia coli K-12 substr. MG1655
<br> Owner: Peter Midford, Created: 05-Apr-2018 17:35:50, Last Modified: 05-Apr-2018 18:05:31, Readable by everyone

In [9]:
#make dictionaries to look up conversion from b-number to common name (or the opposite way, useful in the Ishii KO case). 

def get_id_dicts(data):
    print('Keep in mind that:')
    print('* The conversion table used was for a differnt K-12 substrain: MG1655')

    #load convertion table
    folder_path_data = '/Users/mariekececilia/Documents/master_thesis_code/Data/'
    id_table_path = folder_path_data + "id_table.txt"

    #only extract the columns with the common names and b-numbers, and rename them
    id_table = pd.read_csv(id_table_path, sep='\t', lineterminator='\n', usecols=range(2), dtype="string") #index_col = 0
    id_table = id_table.rename(columns = {'Accession-1': 'b_number', 'Gene Name' : 'common_name'})

    #remove rows with NAN b-numbers
    id_table = id_table[id_table['b_number'].notna()]

    #remove rows with b-numbers not present in the data-set
    id_table = id_table[id_table['b_number'].map(lambda x: True if x in data.columns else False)]
    
    #remove duplicates of common-names:
    duplicated_common_names = id_table[id_table['common_name'].duplicated(keep = False)]
    if (len(duplicated_common_names)>0):
        print('* only one of these duplicated common names were kept:')
        print(duplicated_common_names)
        to_drop = id_table[id_table['common_name'].duplicated()].index.tolist()
        id_table = id_table.drop(to_drop, axis = 'index')
    
    #remove duplicates of b-numbers:
    duplicated_numbers = id_table[id_table['b_number'].duplicated(keep = False)]
    if (len(duplicated_numbers)>0):
        print('* only one of these duplicated b numbers were kept:')
        print(duplicated_numbers)
        to_drop = id_table[id_table['b_number'].duplicated()].index.tolist()
        id_table = id_table.drop(to_drop, axis = 'index')

    #set index to common name
    id_table = id_table.set_index('common_name')
        
    #add missing b-numbers so that they can be looked up
    print("* 'Following b-numbers had no match in the conversion table and are mapped to 'unsure' names:'")
    count = 1
    for number in data.columns:
        if number not in id_table['b_number'].tolist():
            name = 'unsure_' + str(count)
            print('\t- ' + number + ' -> ' + name)
            id_table.loc[name] = number
            count = count+1
    if count == 1:
        print('\t- None')

    #make dictionary from dataframe with common name (index) as key
    common_name_dict = id_table['b_number'].to_dict()
    #swap index and column to make a dictionary with b-number as key instead
    b_number_dict = id_table.reset_index().set_index('b_number')['common_name'].to_dict()    
    return [common_name_dict, b_number_dict]

## ML with LOOCV (leave one out cross validation)

In [10]:
#from sklearn.model_selection import LeaveOneOut

#get the order of which samples were left out during LOOCV
def get_loo_order(X):
    loo = LeaveOneOut()
    test_labels = []
    #loop through arries of train and test indicies
    for train, test in list(loo.split(X)):
        #extract the test label through the test index
        test_labels.append(X.index.tolist()[test[0]])
    return test_labels

### Select by coefficiant of variance

In [11]:
from sklearn.base import BaseEstimator, TransformerMixin #must import to allow construction

#a class that select features based on a coefficient of variation threshold
class CoVSelector(BaseEstimator, TransformerMixin ):

    #Class Constructor 
    def __init__(self, p):
        self.p = p
        self._feature_names = None 

    #Fit to data: find labels of features with high enough cov
    def fit( self, X, y = None ):
        cov = X.std()/X.mean()
        self._feature_names = cov[cov> self.p].index
        return self 

    #Transform data: extract features identified under fitting
    def transform( self, X, y = None):        
        return X[ self._feature_names ]
    
    #Method to make it similar to SelectKBest
    def get_feature_names_out(self, X = None):
        return self._feature_names

### Get model scores (evaluation)

In [12]:
#from sklearn.model_selection import cross_val_score
#from sklearn.model_selection import LeaveOneOut
#import numpy as np

"""
compare MAPE when using different models (pipelines) on the same set of features
models is a dict of {names: corresponding models (pipelines)}
"""
def compare_models(models, X, y, description = 'Models'):
    # create df to store all cv scores of each model
    test_labels = get_loo_order(X)
    cv_results =  (pd.DataFrame(index = test_labels)
                  .rename_axis('Test set')
                  .rename_axis(description, axis='columns'))
    
    # store summaries of ech model
    names, means, stds = [], [], []
    
    for name, model in models.items():
        # evaluate model with CV and store scores as new column
        scores = evaluate_model(model, X, y)
        cv_results[name] = scores
        # add summaries to lists
        [mean, std] = summarise_scores(scores)
        names.append(name)
        means.append(mean)
        stds.append(std)
        
    #create summary df from lists
    summary = pd.DataFrame(data = zip(means, stds), 
                           index = names,
                           columns = ['average', 'std']).rename_axis(description)
    return [cv_results, summary]

def evaluate_model(model, X, y):
    loo = LeaveOneOut()
    scores = cross_val_score(model, X, y, 
                             scoring = 'neg_mean_absolute_percentage_error',
                             cv=loo, error_score='raise') 
    scores = scores * (-1) #counteract the negative MAPE scores
    return scores

#get mean and std of CV scores
def summarise_scores(scores):
    mean = np.mean(scores)
    std = np.std(scores)
    return [mean, std]

### Get model predictions

In [13]:
#from sklearn.model_selection import cross_val_predict
#from sklearn.model_selection import LeaveOneOut

"""
Get predictions of each test sample in LOOCV for different models (pipelines) on the same set of features
models is a dict of {names: corresponding models (pipelines)}
"""
def get_models_preds(models, X, y):
    #find loo order (order of predicted samples)
    preds = pd.DataFrame(index = get_loo_order(y))
    #add actual values
    preds['actual'] = y #rises error if index do not match loo order
    #get and add predictions of each model
    for name, m in models.items():
        pred = prediction_from_model(m, X, y)
        preds[name] = pred
    return preds

#get all predictions from a model (pipeline) with loocv
def prediction_from_model(model, X, y):
    loo = LeaveOneOut()
    pred = cross_val_predict(model, X, y, cv=loo) 
    return pred

### Get the selected features

In [14]:
# ------------------------------------------------------------------------
#        Extract selected features from the different CV iterations
# ------------------------------------------------------------------------
"""
 Get selected features and their corresponding coefficients, returned in two dicts: 
   [sample left out] -> list of features selected when that sample is left out}
   [sample left out] -> list of coefficients of features selected when that sample is left out}
   
   must do the CV manually in order to extract the selected features
"""
def get_loo_selected_feature_names_and_coef(pipeline, X, y):
    
    #find order of test samples left out
    loo_order = get_loo_order(X)
    
    #get selected features and their coefficients when sample left out
    selected_dict = dict()
    coef_dict = dict()
    for test_sample in loo_order:
        
        #fit with split data
        X_train = X.drop(test_sample)
        y_train = y.drop(test_sample)
        fitted = pipeline.fit(X_train,y_train)
        
        #get selecetions and coef
        selected_dict[test_sample] = fitted[:-1].get_feature_names_out()
        coef_dict[test_sample] = fitted[-1].coef_
        
    return selected_dict, coef_dict

# ------------------------------------------------------------------------
#  Create dataframes of selected features in the different CV splits
# ------------------------------------------------------------------------

"""
Create dataframes from the features-coef pairs for each split, ordered
by their importance. The lower the order, the higher absolute coefficant.
One method to get all individual dataframes and one to get a merged
dataframe that allows more analysis (times selected etc.). 
"""

# create dataframe of selected features and their coefficients, one for each CV split
def get_loo_selected_features_and_coef_df(selected_dict, coef_dict):
    
    df_dict = dict()
    
    for test_sample in selected_dict.keys():
        features = selected_dict[test_sample]
        coef = coef_dict[test_sample]
        df = pd.DataFrame([features, coef], index = ['feature', 'coef']).T
        df = order_selected_features_by_coef(df)
        df_dict[test_sample] = df
    return df_dict

# create one dataframe of selected features and their coefficients in all CV splits
def get_loo_selected_features_and_coef_df_merged(selected_dict, coef_dict):
        
    selected = []
    first = True
    
    #turn each set of features and coef into df
    for test_sample in selected_dict.keys():
        features = selected_dict[test_sample]
        coef = coef_dict[test_sample]
        df = pd.DataFrame([features, coef], index = ['feature', 'coef']).T
        df = order_selected_features_by_coef(df)
        
        #merge with the other dataframes
        if first:
            cols = df.columns.tolist()
            df['sample left out'] = test_sample
            selected = df
            first = False
        else:
            df['sample left out'] = test_sample
            selected = pd.concat([selected, df])
            
    return selected[(['sample left out']+cols)]

#add a order column and sort by this one
def order_selected_features_by_coef(df):
    #sort by abs value
    df['abs'] = df['coef'].abs()
    df = df.sort_values(by = 'abs', ascending = False)
    #re-index to get index by sorted order
    df = df.reset_index()
    df['order'] = df.index
    #put original index back
    df = df.set_index('index')
    return df.drop('abs', axis = 1)

# ------------------------------------------------------------------------
#   Analyse number/set of selected features in the different CV splits
# ------------------------------------------------------------------------    
"""
Get list of features in intersection and list of features in union of
slected features (one set of selected features per CV iteration)
"""
def get_selected_features_intersection_and_union(selected_dict):
    #make list of list
    all_subsets= []
    for value in selected_dict.values(): 
        all_subsets.append(value)
        
    #get intersection and union of all lists in list of lists
    intersection = set(all_subsets[0]).intersection(*all_subsets)
    union = set(all_subsets[0]).union(*all_subsets)

    return dict({'intersection': intersection, 'union': union})

## Visualizations

### Remove empty subplots

In [15]:
#import matplotlib.pyplot as plt
#import math

#remove empty subplots 
#NB! will remove the bottom plots and therefore the shared x-axis
# -> cannot use sharex = True, but follow up with remove_xaxes (below)
def remove_empty_subplots(fig, axes, nplots, nrows, ncols):
    empty_plots = nrows*ncols-nplots
    empty_rows = math.floor(empty_plots/ncols)
    empty_cols = empty_plots-empty_rows*ncols
    
    if(empty_rows>0):
        for i in range(nrows-empty_rows, nrows):
            for j in range(ncols):
                fig.delaxes(axes[i][j])
    if(empty_cols>0):
        for j in range(ncols-empty_cols, ncols):
            fig.delaxes(axes[nrows-empty_rows-1][j])
    return fig

#removes x-axis from all but the last non-empty subplots in each column
def remove_xaxes(axes, nplots, nrows, ncols):
    empty_plots = nrows*ncols-nplots
    empty_rows = math.floor(empty_plots/ncols)
    empty_cols = empty_plots-empty_rows*ncols
    last_row = nrows-empty_rows-1
    
    if((empty_cols>0) & (last_row>0)):
        for i in range(0, last_row):
            if i == last_row-1:
                for j in range(0, ncols-empty_cols):
                    #axes[i][j].set_xticks([])
                    axes[i][j].set_xticklabels([])
                    axes[i][j].set_xlabel('')
            else:
                for j in range(0, ncols):
                    #axes[i][j].set_xticks([])
                    axes[i][j].set_xticklabels([])
                    axes[i][j].set_xlabel('')
    if ((empty_cols == 0) & (last_row>0)):
        for i in range(0, last_row):
            for j in range(0, ncols):
                #axes[i][j].set_xticks([])
                axes[i][j].set_xticklabels([])
                axes[i][j].set_xlabel('')

### Box plots of scores

In [16]:
#import matplotlib.pyplot as plt

#draw boxplot subplots with some added defaults
def draw_box_subplots(cv_results, 
                      names, 
                      ax = None, 
                      xlab = 'Model', 
                      ylab = 'MAPE',
                      showmeans=True,
                      patch_artist=True,
                      boxprops=dict(facecolor='lightblue', 
                                    color='lightblue'),
                      meanprops = dict(markeredgecolor='darkgreen', 
                                       markerfacecolor='darkgreen', 
                                       markersize = '5'),
                      #medianprops=dict(color='purple'),
                      **kwargs):
    ax.boxplot(cv_results, 
               labels=names,
               showmeans=showmeans,
               patch_artist = patch_artist,
               boxprops=boxprops,
               meanprops = meanprops,
               #medianprops=medianprops,
               **kwargs)
    ax.set_ylabel(ylab)
    ax.set_xlabel(xlab)
    return ax

### Predicted vs actual values

In [17]:
#import seaborn as sns
#import matplotlib.pyplot as plt

#draw scatter plots of Actual values vs predicted values (as subplots)
def plot_preds(x, y, data, min_lim = 0, max_lim = 4, ax=None, diagonal = True, **kwargs):
    if (diagonal):
        ax.plot([min_lim, max_lim], [min_lim,max_lim], linestyle = 'dotted', color = 'grey', alpha = 0.5)
    sns.scatterplot(x=x, y=y, data=data, ax = ax, **kwargs)
    ax.set_title(x)
    ax.set_ylabel('Actual value')
    ax.set_xlabel('Predicted value')
    ax.set_xlim(min_lim, max_lim)
    ax.set_ylim(min_lim, max_lim)
    if (ax.get_legend() != None):
        ax.get_legend().remove()
    return ax